<a id='top'></a>

# CSCI 3022: Intro to Data Science - Spring 2020 Practicum 2
***

This practicum is due on Canvas by **11:59 PM on Friday May 1**. Your solutions to theoretical questions should be done in Markdown/MathJax directly below the associated question.  Your solutions to computational questions should include any specified Python code and results as well as written commentary on your conclusions.  

**Here are the rules:** 

1. All work, code and analysis, must be your own. 
1. You may use your course notes, posted lecture slides, textbooks, in-class notebooks, and homework solutions as resources.  You may also search online for answers to general knowledge questions like the form of a probability distribution function or how to perform a particular operation in Python/Pandas. 
1. This is meant to be like a coding portion of your midterm exam. So, the instructional team will be much less helpful than we typically are with homework. For example, we will not check answers, help debug your code, and so on.
1. If something is left open-ended, it is because we want to see how you approach the kinds of problems you will encounter in the wild, where it will not always be clear what sort of tests/methods should be applied. Feel free to ask clarifying questions though.
2. You may **NOT** post to message boards or other online resources asking for help.
3. You may **NOT** copy-paste solutions *from anywhere*.
4. You may **NOT** collaborate with classmates or anyone else.
5. In short, **your work must be your own**. It really is that simple.

Violation of the above rules will result in an immediate academic sanction (*at the very least*, you will receive a 0 on this practicum or an F in the course, depending on severity), and a trip to the Honor Code Council.

**By submitting this assignment, you agree to abide by the rules given above.**

***

**Name**:  Sahand Setareh

***


**NOTES**: 

- You may not use late days on the practicums nor can you drop your practicum grades. 
- If you have a question for us, post it as a **PRIVATE** message on Piazza.  If we decide that the question is appropriate for the entire class, then we will add it to a Practicum clarifications thread. 
- Do **NOT** load or use any Python packages that are not available in Anaconda 3.6. 
- Some problems with code may be autograded.  If we provide a function API **do not** change it.  If we do not provide a function API then you're free to structure your code however you like. 
- Submit only this Jupyter notebook to Canvas.  Do not compress it using tar, rar, zip, etc. 
- This should go without saying, but... For any question that asks you to calculate something, you **must show all work to receive credit**. Sparse or nonexistent work will receive sparse or nonexistent credit.


---

In [1]:
from scipy import stats
from math import isnan
import numpy as np 
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from calendar import month_name, different_locale

<br>

---

### [50 points] Problem 1: Multiple Linear Regression to Explain House Hauntings

<img src="https://s-media-cache-ak0.pinimg.com/originals/09/72/01/09720128cff5de4d4af038cd3fcf7f69.jpg" style="width: 300px;"/>

In an effort to control the skyrocketing prices of real estate in the Colorado Front Range, Governor Polis implemented a cutting edge new intervention. This new program oversaw the introduction of ghosts back into their natural ecosystem, after the ghost population seriously dwindled in recent decades due to overhaunting. However, an unfortunate miscalculation has led to haunted houses becoming a very serious problem in Colorado. Modern problems require modern solutions, so Governor Polis has hired you and the famous hedgehog data scientist/part-time ghostbuster Amy to determine what features of a house may be used to best predict a `haunted` score, related to the probability that a house with the given features is haunted (higher $\leftrightarrow$ more likely to be haunted).

You decide to use multiple linear regression to understand and predict what factors lead to increased haunted house hazard. You collected a data set from Haunted Zillow, the lesser-known database of haunted house prices and attributes. The data cover a variety of potential features, and you'll find this data in the file `houses.csv`. 

**Response**: 

- $\texttt{haunted}$: a haunting score, related to the probability that a house with the given features is haunted (higher $\leftrightarrow$ more likely to be haunted)

**Features**: 

- $\texttt{age}$: age of the house, in years
- $\texttt{area}$: square footage of interior of house
- $\texttt{bathrooms}$: number of bathrooms
- $\texttt{distance metro}$: distance to the nearest major metropolitan area (in miles)
- $\texttt{distance cemetery}$: distance to the nearest cemetery (in miles)
- $\texttt{cats}$: the number of cats within a one-block radius of the house
- $\texttt{howls}$: the number of wolf howls heard on an average night in the house's neighborhood
- $\texttt{clouds}$: what percentage of the sky was covered by clouds (fraction, 0-1)
- $\texttt{precipitation}$: amount of precipitation in the past 72 hours (inches)
- $\texttt{misery index}$: an economic indicator for how miserable the average United States citizen is, based on the unemployment rate and the inflation rate. More [here](https://www.stuffyoushouldknow.com/podcasts/whats-the-misery-index.htm) and [here](https://en.wikipedia.org/wiki/Misery_index_(economics)). Higher values correspond to more miserable citizens.
- $\texttt{ice cream sold}$: the number of units of ice cream sold at the farmer's market the week the house was most recently sold

**Part A**: Read the data from `houses.csv` into a Pandas DataFrame.  Note that since we will be doing a multiple linear regression we will need all of the features, so you should drop any row in the DataFrame that is missing data. 

In [2]:
# Your code here.
df = pd.read_csv("/Users/sahandsetareh/Spring 2020/CSCI 3022/Practicum2/houses.csv")
df = df.dropna()
df

,age,area,bathrooms,distance metro,distance cemetery,cats,howls,clouds,precipitation,misery index,ice cream sold,haunted
0,118.94,2113.0,4,7.1,3.23,7.0,3.0,1.00,0.82,12.99,273.0,0.123131
1,20.60,1773.0,2,7.4,4.19,5.0,5.0,1.00,0.99,16.77,184.0,-0.200243
2,260.84,2511.0,3,7.0,3.38,2.0,0.0,1.00,1.17,16.49,141.0,0.258651
3,101.33,2000.0,2,7.9,4.05,6.0,8.0,0.13,0.92,8.28,146.0,0.147983
4,96.94,1476.0,1,7.5,5.75,4.0,1.0,1.00,1.73,5.90,178.0,-0.113756
...,...,...,...,...,...,...,...,...,...,...,...,...
67,237.66,1403.0,2,6.6,5.97,8.0,2.0,1.00,2.16,6.85,271.0,0.352572
68,65.39,1880.0,4,4.1,3.59,7.0,2.0,0.60,1.67,16.83,211.0,0.331913
69,56.24,1996.0,2,7.4,3.38,4.0,4.0,1.00,0.71,6.42,215.0,0.627473
70,4.87,1732.0,1,7.1,3.96,7.0,2.0,1.00,1.46,19.44,122.0,0.411308


**Part B**: Perform the appropriate statistical test at the $\alpha = 0.01$ significance level to determine if _at least one_ of the features is related to the the response $y$.  Clearly describe your methodology and show all computations in Python. 

Performing an F-test with the following hypothesis: 

$$H_0: \beta_1 = \beta_2 = \beta_3 = ... = \beta_{11} = 0$$
$$H_1: \beta_i \neq 0, i = 1,2,3,...,11 $$

In [3]:
# Your code here.

# We check to see if at least one of the features is related to response y ('haunted')
y = df["haunted"]

# Load feature responses other than haunted
x = df.loc[ : , df.columns != "haunted"]
x = sm.add_constant(x)

# we read in our model using the response y as one parameter and the other feautures as x
full_model = sm.OLS(y, x).fit()

# printing a summary of our model
full_model.summary()

/Users/sahandsetareh/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                haunted   R-squared:                       0.748
Model:                            OLS   Adj. R-squared:                  0.702
Method:                 Least Squares   F-statistic:                     16.22
Date:                Fri, 01 May 2020   Prob (F-statistic):           3.75e-14
Time:                        23:49:24   Log-Likelihood:                 4.2800
No. Observations:                  72   AIC:                             15.44
Df Residuals:                      60   BIC:                             42.76
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.3097      0.342      0.905      0.369      -0.375       0.994
age                   0.0012      0.000      3.267      0.002       0.000       0.002
area               3.573e-05    8.1e-05      0.441      0.661      -0.000       0.000
bathrooms            -0.0104      0.031     -0.330      0.743      -0.073       0.052
distance metro       -0.0173      0.036     -0.477      0.635      -0.090       0.055
distance cemetery    -0.1149      0.010    -11.107      0.000      -0.136      -0.094
cats                  0.0370      0.011      3.343      0.001       0.015       0.059
howls                -0.0091      0.013     -0.694      0.491      -0.035       0.017
clouds               -0.1022      0.139     -0.738      0.463      -0.379       0.175
precipitation         0.0166      0.076      0.217      0.829      -0.136       0.169
misery index          0.0051      0.006      0.802      0.426      -0.008       0.018
ice cream sold        0.0004      0.001      0.788      0.434      -0.001       0.002
==============================================================================
Omnibus:                        0.027   Durbin-Watson:                   1.926
Prob(Omnibus):                  0.986   Jarque-Bera (JB):                0.176
Skew:                          -0.026   Prob(JB):                        0.916
Kurtosis:                       2.764   Cond. No.                     2.36e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.36e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Our model gives us an F-statistic $F = 16.22$ and a p-value $p = 3.75 x 10^{-14}$. We can see that $ p < \alpha = 0.01$, and as such we reject the null hypothesis. There is evidence that $\beta \neq 0$, so at least one of the provided features must be related to the response given. 

**Part C**: Write a function `forward_select(df, resp_str, maxk)` that takes in the DataFrame, the name of the column corresponding to the response, and the maximum number of desired features, and returns a list of feature names corresponding to the `maxk` most important features via forward selection.  At each stage in forward selection you should add the feature whose inclusion in the model would result in the lowest sum of squared errors $(SSE)$. Use your function to determine the best $k=5$ features to include in the model. Clearly indicate which feature was added in each stage. 

**Note**: The point of this exercise is to see if you can implement **foward_select** yourself.  You may of course use canned routines like statmodels OLS, but you may not call any Python method that explicitly performs forward selection.

In [4]:
# Your code here.
def forward_select(df, resp_str="haunted", maxk=3):
    
    # response y  
    y = df[resp_str]
    
    # get features we will potentially use 
    remaining_features = list(df.columns[df.columns != resp_str])
    
    # features that we will keep
    features_keep = []
    
    # forward selection procedure
    # iterate through stages
    for i in range(maxk):
        
        # SSE value
        sumsquared_error = []
        
        # Iterate through the remaining features, create model based on forward selection parameters 
        for feature in remaining_features:
            
            # x is a dataframe of the features we keep and the next one in the list of our potential features
            x = df[features_keep + [feature]]
            x = sm.add_constant(x)
            
        # Model based on our response y and dataframe of features x
            model = sm.OLS(y, x).fit() 
            
        # Add SSE value to our list
            sumsquared_error.append(np.sum((y - model.predict(x))**2))
        
        # Grab the minimum SSE
        new_feature = remaining_features[np.argmin(sumsquared_error)]
        
        # Adding the new feature to the features we will keep 
        features_keep = features_keep + [new_feature]
        
        # Remove the featurre once we have added it above
        remaining_features.remove(new_feature)
        
        # Printing the stage and the features added
        print("Stage " + str(i) + " - " + "added " + str(new_feature))
        
        # return the list of features
    return features_keep
              
ideal_features = forward_select(df, resp_str = "haunted", maxk = 5)

Stage 0 - added distance cemetery
Stage 1 - added cats
Stage 2 - added age
Stage 3 - added misery index
Stage 4 - added distance metro


**Part D**: Write down the multiple linear regression model, including estimated parameters, obtained by your forward selection process. 

In [5]:
# Your code here.

# reduced model done using the features we obtained in the forward selection process in part C
red_x = df.loc[ : , ideal_features]
red_x = sm.add_constant(red_x)

# Using the response 'haunted'
y = df["haunted"]
red_mod = sm.OLS(y, red_x).fit()

# printing a summary of our model
red_mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                haunted   R-squared:                       0.743
Model:                            OLS   Adj. R-squared:                  0.723
Method:                 Least Squares   F-statistic:                     38.12
Date:                Fri, 01 May 2020   Prob (F-statistic):           3.40e-18
Time:                        23:49:24   Log-Likelihood:                 3.4850
No. Observations:                  72   AIC:                             5.030
Df Residuals:                      66   BIC:                             18.69
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.3848      0.240      1.607      0.113      -0.093       0.863
distance cemetery    -0.1114      0.009    -12.215      0.000      -0.130      -0.093
cats                  0.0348      0.010      3.398      0.001       0.014       0.055
age                   0.0012      0.000      3.529      0.001       0.001       0.002
misery index          0.0058      0.006      0.962      0.339      -0.006       0.018
distance metro       -0.0269      0.032     -0.830      0.409      -0.092       0.038
==============================================================================
Omnibus:                        0.218   Durbin-Watson:                   1.957
Prob(Omnibus):                  0.897   Jarque-Bera (JB):                0.410
Skew:                          -0.034   Prob(JB):                        0.815
Kurtosis:                       2.636   Cond. No.                     1.38e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.38e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Multiple linear regression model: $\hat{y} = (0.3848) + (-0.1114)$ (distance cemetery) $ + (0.0348) $(cats) $ + (0.0012) $(age) $ + (0.0058) $(misery index) $ + (0.=0.0269) $ (distance metro)

**Part E**: Perform the appropriate statistical test at the $\alpha = 0.05$ significance level to determine whether there is a statistically significant difference between the full model with all features and the reduced model obtained by forward selection in **Part D**. 

$H_{0}: \beta_{i} = 0$, $i = $ distance cemetary, cats, age, misery index, deistance metro

$H_{1}: $ For at least one, $\beta_{i} \neq 0$, $i = $ distance cemetary, cats, age, misery index, distance metro, misery index, ice cream sold

Performing a partial F-Test:

$$F = \frac{\frac{SSE_{reduce} - SSE_{full}}{p - k}}{\frac{SSE_{full}}{n - p - 1}} $$

We analyze the rejection region: $ F \geq F_{\alpha, p, n - p - 1}$

We analyze the p-value: if $p < \alpha$, reject the null hypothesis

In [6]:
# Your code here.

# Parameters for alpha, k, n and p
alpha = 0.05
p = len(df.columns)-1
k = len(ideal_features) 
n = len(df)

# reduced and full sum squared errors
reduced_sse = np.sum((y - regression_mod.predict(regression))**2)
full_sse = np.sum((y - full_model.predict(x))**2)

# Calculating F value
f = ((reduced_sse - full_sse) / (p - k))/((full_sse) / (n - p - 1))

# F-alpha to compare against F
f_alpha = stats.f.ppf(1 - alpha, p - k, n - p - 1)

# P-Value
pval = 1 - stats.f.cdf(f, p - k, n - p - 1)

print("Sum Squared Error (Reduce) = " + str(round(sumsquared_error_red, 3)))
print("Sum Squared Error (Full) = " + str(round(full_sse, 3)))
print()
print("F = " + str(round(f, 3)) + ", F-alpha = " + " " + str(alpha_f))
print("P-value = " + str(pval))

NameError: name 'regression_mod' is not defined

Because our P-Value is less than alpha, $p > \alpha$, we fail to reject the null hypothesis. This indicates that the reduced model is better than the reduced model (statistically speaking, it is superior in this instance).

**Part F**: Based on your conclusions in **Part E**, use the _better_ of the two models to predict the haunted house hazard when the following features are observed: 

- $\texttt{age}$: 150 years
- $\texttt{area}$: 2200 square feet
- $\texttt{bathrooms}$: 3 bathrooms
- $\texttt{distance metro}$: 25 miles
- $\texttt{distance cemetery}$: 5 miles
- $\texttt{cats}$: 20 cats
- $\texttt{howls}$: 5 wolf howls/night
- $\texttt{clouds}$: 0.65 cloud cover
- $\texttt{precipitation}$: 0 inches
- $\texttt{misery index}$: 10
- $\texttt{ice cream sold}$: 125

In [ ]:
# Your code here.

features = { "const" : 1, 
            "age" : 150, 
            "area" : 2200, 
            "bathrooms" : 3, 
            "distance metro" : 25,
            "distance cemetery" : 5, 
            "cats": 20, 
            "howls": 5, 
            "clouds" : 0.65, 
            "precipitation" : 0,
            "misery index" : 10, 
            "ice cream sold" : 125 }

# Y-hat value we will determine using our reduced model

yh = 0

for i, j in zip(red_mod.params.index, red_mod.params):
    yh += j * features[i] 
    
print("Y-hat value = " + str(round(yh, 3)))

**Part G:** Governor Polis dabbles a bit in the art of data science, as well as the science of data art. He tells you that the response (`haunted` score) that you and Amy predicted is actually the natural logarithm of the _odds_ that a house with the given features is haunted, where if $p$ is the probability that a house is haunted, then the odds are given by $$\text{odds} = \dfrac{p}{1-p}$$

So using Govenor Polis's information and combining it with our knowledge of logistic regression, the probability of a house being haunted is:

$$p(\text{haunted} \mid x_1, x_2, \ldots) = \dfrac{1}{1+\texttt{exp}[-(\beta_0 + \beta_1x_1 + \ldots + \beta_p x_p)]}$$

Perform this computation, then use a decision threshold of 0.5 to classify the house from **Part F** as haunted or not haunted. No new models should be fit here; use the same model that you used in Part F.

In [ ]:
# Your code here.

# Function to compute the proability
def sigmafun(yh):
    return 1 / (1 + np.exp(-yh))

p = sigmafun(yh)

print("P = " + str(round(p, 3)))

We can observe that because the value of $p$ is greater than that of the decision threshold (0.5), the house can be classified as being haunted!

### [50 points] Problem 2: Amazon Forest Fires
A non-profit trying to protect the amazon rain forest has recruited you to join their data science corps. For your first task, they've given you a dataset with the number of reported forest fires in each state in the Amazon region of Brazil during each month between 1998 and 2017. The Brazilian government has 500 extra wildland firefighters and they have asked your non-profit to determine which state or states they should allocate these firefighters to during each month of the year. To do this, they want you to calculate an 80% confidence interval for the mean and median number of fires that occur during each month for each state, and use those statistics to determine where the firefighters should be assigned.

#### Part A: Loading The CSV
Read the csv located in `amazon.csv` into a pandas data frame. Brazil and many other countries use the period (.) symbol as a thousands separator and a comma (,) as the decimal separator. Ex. One Thousand And $\frac{75}{100}$ would be represented as $1.000,75$ instead of the familiar english notation $1,000.75$. When you read it in, you'll need to use a period(.) as the thousands separator and a comma(,) as the decimal separator. Because the comma is already in use as the decimal separator, this file uses a different character to separate columns in the data. Open up the file in a text editor and figure out what character was used. Then find the correct arguments to `pd.read_csv` to read in this file properly. Look up the docs if you're unsure what the arguments you'll need are. Print out the `.info` summary of the dataframe after you've read it in.

In [ ]:
amazon_fires = pd.read_csv('/Users/sahandsetareh/Spring 2020/CSCI 3022/Practicum2/amazon.csv', delimiter = ";", thousands = '.', decimal = ',') # Your code here.
amazon_fires.info()

#### Part B: Data Cleaning

This dataset isn't paticularly useful in it's current state, so we'll need to clean it up a bit. Some data scientists say that most of their job is to wrangle data, so this will give you a taste of cleaning a real world data set. Perform the following tasks. 
1. Drop the 'date' column. The only information this column holds is the year, which we already have in another column. Use the `.info` summary provided to check your work.
2. Drop any rows with null values in any of the remaining columns. Use the provided code to print the number of rows remaining after this step.
3. Print all the unique values of the 'month' column. You'll notice that one is encoded with a different character encoding then the format that pandas is using.
4. Convert the Portugese month names to English month names. If you'd like to use them, we've included the 'month_name' and the 'different_encoding' modules of the python calendar library. There are many ways to accomplish this task, and these modules are not required, but may make things easier. As part of this step, you should make sure that the Portugese month with the encoding problem is translated to the correct english month. Use the `.unique` method provided for you to check your work. 
5. Check the number column for any values that seem impossible. If you find any values you think are impossible, drop them. As a guidline, we would never expect a single state to have more than 50,000 reported forest fires in a single month. Also keep in mind that we are tracking forest fires here. Do negative or fractional forest fires really make sense? You should check for any obivously impossible conditions that you think might occur, and drop rows accordingly. Use the provided code to print the number of rows remaining after this step.
6. Since you're new on the job, some of your co-workers may have played a prank on you... Print out all the unique values of the 'year' column and drop any rows with values that don't make sense. Use the provided code to print the number of rows remaining after this step.
7. For every state in the data, print the number of rows the state has associated with it. A number of states have far more observations than the others. Each state should have roughly 240 observations (20 years multiplied by 12 months/year minus any bad data). Drop all the observations for any states that have more than 240 rows associated with them. For two points of extra credit, figure out why these states have way more rows associated with them than they should. If you choose to do the extra credit, put your answer in the markdown cell below. 
8. To give you an idea of whether your answer is correct, we've provided a unit test below the last cell. It should pass. If it doesn't, go back and figure out which step has gone awry.

We've given you a code cell for each task to make organizing the grading a bit easier. Please perform step 1 in the first code cell and so on.

**NOTE:** Since some of these tasks are not totally trivial, you may use any resources other than your classmates on this part of this problem. This means you may consult google, stack overflow, the python/pandas documentation, some random book on pandas you might have, etc... But you may not consult your classmates for help. We will also be more helpful on this problem in office hours and in response to your *private* piazza messages.  ***CITE ALL RESOURCES USED IN A CODE COMMENT. A URL OR A BOOK TITLE IS SUFFICIENT. ANY CODE OBIVOUSLY COPIED FROM OUTSIDE SOURCES WITH OUT A CITATION WILL EARN YOU NO CREDIT ON THIS PROBLEM.***

In [ ]:
#Code for data cleaning task 1 here:

# Dropping the date column
amazon_fires = amazon_fires.drop(['date'], axis = 1)
amazon_fires.info()

In [ ]:
#Code for data cleaning task 2 here:

# Droping any null values from the four columns
amazon_fires = amazon_fires.dropna(subset = ['year', 'state', 'month', 'number'])

# The number of rows we should have
print(len(amazon_fires))

In [ ]:
#Code for data cleaning task 3 here:

# Listing out the names of the months (in Portuguese)
uniqueMonths = amazon_fires.month.unique()
uniqueMonths

In [ ]:
#Code for data cleaning task 4 here:

# Using an online reference (linked below), I manually changed all of the Portuguese names to English

amazon_fires.loc[amazon_fires['month'] == 'Janeiro',   ['month']] = 'January'
amazon_fires.loc[amazon_fires['month'] == 'Fevereiro', ['month']] = 'February'
amazon_fires.loc[amazon_fires['month'] == 'Mar�o',     ['month']] = 'March'
amazon_fires.loc[amazon_fires['month'] == 'Abril',     ['month']] = 'April'
amazon_fires.loc[amazon_fires['month'] == 'Maio',      ['month']] = 'May'
amazon_fires.loc[amazon_fires['month'] == 'Junho',     ['month']] = 'June'
amazon_fires.loc[amazon_fires['month'] == 'Julho',     ['month']] = 'July'
amazon_fires.loc[amazon_fires['month'] == 'Agosto',    ['month']] = 'August'
amazon_fires.loc[amazon_fires['month'] == 'Setembro',  ['month']] = 'September'
amazon_fires.loc[amazon_fires['month'] == 'Outubro',   ['month']] = 'October'
amazon_fires.loc[amazon_fires['month'] == 'Novembro',  ['month']] = 'November'
amazon_fires.loc[amazon_fires['month'] == 'Dezembro',  ['month']] = 'December'

print(amazon_fires['month'].unique())

Source used for Month names: http://www.portugueselanguageguide.com/vocabulary/months.asp

In [ ]:
#Code for data cleaning task 5

# Drop rows from the dataframe that exhibit unrealistic numbers per the guidelines

amazon_fires = amazon_fires.drop(amazon_fires[(amazon_fires.number > 50000)].index)
amazon_fires = amazon_fires.drop(amazon_fires[(amazon_fires.number < 0)].index)
amazon_fires = amazon_fires.drop(amazon_fires[(amazon_fires.number % 1 != 0)].index)

# print number of rows remiaing
print(len(amazon_fires))

In [ ]:
#Code for data cleaning task 6

# amazon_fires.year.unique() reveals the mischievous prank, and we removed these values from that column 

amazon_fires = amazon_fires.drop(amazon_fires[(amazon_fires.year == '1000bc')].index)
amazon_fires = amazon_fires.drop(amazon_fires[(amazon_fires.year == '-40')].index)
amazon_fires = amazon_fires.drop(amazon_fires[(amazon_fires.year == '10bc')].index)
amazon_fires = amazon_fires.drop(amazon_fires[(amazon_fires.year == 'our new data scientist won\'t notice this')].index)

# print number of rows remiaing
print(len(amazon_fires))

In [ ]:
#Code for data cleaning task 7

# I used the following to print out value counters per state, then manually remove the three that 
# had counts greater than 240 

# print(amazon_fires['state'].value_counts())

amazon_fires = amazon_fires.drop(amazon_fires[(amazon_fires.state == 'Rio')].index)
amazon_fires = amazon_fires.drop(amazon_fires[(amazon_fires.state == 'Mato Grosso')].index)
amazon_fires = amazon_fires.drop(amazon_fires[(amazon_fires.state == 'Paraiba')].index)

# Checking to verify that only states with no more than 240 observations remain
print(amazon_fires['state'].value_counts())

Extra Credit: I would think that a reason why these states would have a lot more rows associated with them than other states are misreportings that happen multiple times per state. Organizational entry-keeping challenges may arise with a large country such as Brazil which must keep track of such extensive events. 

In [ ]:
#Given Test. This Cell Should Not Throw an Exception!
assert \
    len(amazon_fires['state'].unique()) == 20 and \
    list(amazon_fires['month'].unique()) == \
        ['January', 'February', 'March', 'April', 'May', 'June',
             'July', 'August','September', 'October', 'November',
             'December'] and \
    len(amazon_fires) == 4772, 'somethings wrong in problem 1.'

#### Part C: Medians and Means!
In this part of the problem, we'll calculate an 80% confidence interval for both the mean and median number of wildfires each state has during each month of the year. 

For the mean you should use the appropriate confidence interval with the correct distribution. Remember to check how many observations we have. Use the sample standard deviation. 

For the median, we'll have to bootstrap it because the median is not known to be normally distributed. You should bootstrap 1000 samples of the same length as the original sample for each month for each state. Calculate the median for each bootstrapped sample. Then take the middle 80% of the bootstrapped medians as your confidnce interval. This is called a bootstrapped percentile median. There are a few more complex and slightly more rigourous ways to estimate the median from bootstrapped samples, but this will serve for our purposes.

You're given a dictionary of dictionaries to store your confidence intervals for the medians and means in. 

Take a look at the dictionary structure below. 

The idea here is that for every month, for every state, you will fill in the `mean_CI` with a length two list that contains the low and high end of the confidence interval for the true mean number of fires for that state in that month. 

Similiarly, for every month, for every state, you will fill in the `median_CI` with a length two list that contains the low and high end of the confidence interval for the true median number of fires for that state in that month.

For example:

When you're done `months['January']['Acre']['mean_CI']` should be a list with the low and high bounds for the confidence interval of the true mean number of wildfires in the state of Acre in January. So `months['January']['Acre']['mean_CI'][0]` should be the low end of the CI for the mean, and `months['January']['Acre']['mean_CI'][1]` should be the high end of the CI for the mean.

`months['January']['Acre']['median_CI']` should hold the confidence interval for the true median number of wildfires in the state of Acre in January. So `months['January']['Acre']['median_CI'][0]` should be the low end of the CI for the median, and `months['January']['Acre']['median_CI'][1]` should be the high end of the CI for the median.

In [ ]:
#GIVEN CODE DO NOT CHANGE THIS!!!
#YOU SHOULD BE WRITING CODE IN THE NEXT CELL(s) THAT FILLS IN THE 'months' DICTIONARY.

#If you're curious what copy and deep copy do and why we used them here see an explanation 
#here: https://thispointer.com/python-how-to-copy-a-dictionary-shallow-copy-vs-deep-copy/

from copy import deepcopy

mean_median_dict ={
    'mean_CI' : None,
    'median_CI': None
}

CI_median_num_fires = {
    'Acre': dict(mean_median_dict),
    'Alagoas':dict( mean_median_dict),
    'Amapa':dict( mean_median_dict),
    'Amazonas':dict( mean_median_dict),
    'Bahia':dict( mean_median_dict),
    'Ceara':dict( mean_median_dict),
    'Distrito Federal':dict( mean_median_dict),
    'Espirito Santo':dict( mean_median_dict),
    'Goias':dict( mean_median_dict),
    'Maranhao':dict( mean_median_dict),
    'Minas Gerais':dict( mean_median_dict),
    'Para':dict( mean_median_dict),
    'Pernambuco':dict( mean_median_dict),
    'Piau':dict( mean_median_dict),
    'Rondonia':dict( mean_median_dict),
    'Roraima':dict( mean_median_dict),
    'Santa Catarina':dict( mean_median_dict),
    'Sao Paulo':dict( mean_median_dict),
    'Sergipe':dict( mean_median_dict),
    'Tocantins':dict( mean_median_dict)  
}

months = {
    'January': deepcopy(CI_median_num_fires),
    'February': deepcopy(CI_median_num_fires),
    'March': deepcopy(CI_median_num_fires), 
    'April': deepcopy(CI_median_num_fires), 
    'May': deepcopy(CI_median_num_fires),
    'June': deepcopy(CI_median_num_fires),
    'July': deepcopy(CI_median_num_fires),
    'August': deepcopy(CI_median_num_fires), 
    'September': deepcopy(CI_median_num_fires), 
    'October': deepcopy(CI_median_num_fires),
    'November': deepcopy(CI_median_num_fires),
    'December': deepcopy(CI_median_num_fires)
}

In [ ]:
#Your code here:
#Your code should modify the 'months' dictionary provided to add the CIs.

# Bootsrapping the median because it is not normally distributed
def boot_medians(sample, month, state):
    bootstrap = np.array([np.median(np.random.choice(sample, replace = True, size = len(sample)))])
    l = np.percentile(bootstrap, 10)
    u = np.percentile(bootstrap, 90)
    months[month][state]['median_CI'] = [l, u]

# Function that determines the confidence interval for our means
def mean_ci(sample, month, state):
    n = len(sample)
    mean = np.mean(sample)
    var = sum((xi - mean)**2 / (n - 1) for xi in sample)
    std = np.sqrt(var)
    t_crit = stats.t.ppf(0.9, df = n - 1)
    l = mean - t_crit * std / np.sqrt(n)
    u = mean + t_crit * std / np.sqrt(n)
    months[month][state]['mean_CI'] = [l, u]
    
# Confidence Intervals for Medians
for month in months:
    print(month)
    for state in CI_median_num_fires:

        medians = amazon_fires[(amazon_fires['month'] == month) & (amazon_fires['state'] == state)].number.tolist()
        for i in range(1000):
            boot_medians(medians, month, state)
        
# Confidence Intervals for Means
for month in months:
    for state in CI_median_num_fires:
        means = amazon_fires[(amazon_fires['month'] == month) & (amazon_fires['state'] == state)].number.tolist()
        mean_ci(means, month, state)


In [ ]:
#DONT CHANGE THIS. WE USE IT TO MAKE THE OUTPUT LEGIBLE FOR GRADING
import pprint
pp = pprint.PrettyPrinter(indent=1)
pp.pprint(months)

In [ ]:
#Given Test for the mean confidence intervals

rounded_mean_CI = [round(x, 2) for x in months['April']['Acre']['mean_CI']]
assert rounded_mean_CI == [0.76, 3.34], 'somethings wrong in the mean'

In [ ]:
#Given test for the median confidence intervals. 
#Your code is probably correct if it passes this test, but since bootstrapping the medain is a stochastic process
#you may have this test fail. If it fails, run it a few times. 
#If it continues to fail, your code is probably incorrect.

low_median_CI = months['April']['Acre']['median_CI'][0]
high_median_CI = months['April']['Acre']['median_CI'][1]
assert -1 <= low_median_CI <= 1 and 0 <= high_median_CI <= 3, 'somethings wrong in the median'

#### Part E: Where Do The Firefighters Go?
Now, we'll determine which state the Brazilian government should assign it's fire fighters to. For each month of the year, you should perform the folllowing selection process:
1. Find the state with the highest CI for the median for this month (it's easiest and ok to just use the upper bound here). 
2. Find any states that have a median CI that overlaps with the highest CI foud in step 1. If no states overlap with the highest CI found in step 1, then use that stat. 
3. If overlapping confidence intervals are found on the median, we'll use the CI for the mean to break ties.
4. Out of the states with overlapping CIs for median (every state in part 3), find the state with the highest mean CI. 
5. Determine if any of the states from part 3 have a mean CI that overlaps with the state found in step 4. 
6. If no state overlap with the state found in part 4, then just use that state. If other states have overlapping mean CIs too, then we'll split up the firefighters and assign some of them to every state that has both an overlapping median and mean CI with the state that has the highest median CI.

Once you've used the selection process above, use a markdown table to display a list of each state that recieves  some of the firefighters for each month.

In [ ]:
# Task 1 - Find the state with the highest upper-bound for the CI of the median

# Lists to hold lower and upper CI values
lower, upper = [], []

# maxVal is a list of size 12 to hold the highest value CI
maxVal = []

# Iterate through months
for month in months:
    # Reset these values for every new month
    max_ci, max_state, max_month = 0, "", ""
    low_ci = 0 
    # Iterate through states
    for state in CI_median_num_fires:
        compare = months[month][state]['median_CI'][1]
        # If compare is larger than the highest CI value we have found - set max_ci to compare's value
        if compare > max_ci:
            max_ci = compare
            # Make note of what state has the maximum value for that month
            max_state = state
        low_ci = months[month][state]['median_CI'][0]
        max_month = month
    maxVal.append((max_month, max_state, max_ci))
    lower.append(low_ci), upper.append(max_ci)
    
print("Task 1: " + "\n")
print("(  Month,   State with highest CI (median),   highest CI (using upper bound))" + "\n")
for i in range(len(maxVal)):
    print(maxVal[i])

# Task 2 -  Make a list of every other state that has an upper-bound for the CI on the median that is greater 
# than or equal to the lower bound of the state found in number 1

# My approach here would have been to have a list, and iterating through every state, compare and append to this list
# every state that has a CI on the median when compared to the lower bound on the state we determined to have 
# the highest CI in Task 1

# Task 3 - Add the state from number 1 to the list from number 2.

# Straightforward task had Task 2 been completed - append to the list from Task 2 the state with the maximum value
# CI from Task 1
    
# Task 4 - Find the state with the highest upper-bound for the mean CI from the list in number 3.

# Again, similar to Task 1, iterate through the list of CI's and make note of the state with the highest upper-bound

# Task 5 - Find all of the other states, from only the list in number 3, that have an overlapping CI on the mean 
# with the state from number 4. This should be every state in the list from number 3 that has an upper bound 
# (on the CI of the mean) greater than lower bound (also CI for mean) of the state in number 4. 
# If this list is empty,  assign the firefighters to the state in number 4.

# Here, I would have determined the overlapping CI on the mean with the state in number 4 by comapring the lower
# value of the CI against the upper bound for the state found in Task 4. If this list returns empty then fire
# fighters are instead allocated to the state in Task 4. 

# Task 6 - Add the state from number 4 to the list from number 5. Split the firefighters among these states.

# Distribute the 500 availible firefighters among the states with the highest CIs
